In [1]:
import numpy as np
import numba as nb
import random
import matplotlib.pyplot as plt

## 4: Multiplying Matrices
- test that $\sigma_j\sigma_k = \delta_{jk}\mathbf{1} + i \epsilon_{jkl}\sigma_l$

In [2]:
#from 2024-1
@nb.jit(nopython=True)
def norm_diff(L, M):
    LminusM = L - M
    mag_LminusM = np.sqrt(np.sum(np.abs(LminusM)**2))
    mag_L = np.sqrt(np.sum(np.abs(L)**2))
    mag_M = np.sqrt(np.sum(np.abs(M)**2))
    
    delta = mag_LminusM / (mag_L + mag_M)
    
    return delta

## 5: Create function to multiply matrices
- (5a): MAKE GENERAL MATRIX MAKER: create function that inputs 2 arrays of complex numbers ($A_0, \vec{A}$) and ($B_0, \vec{B}$) and output array of complex numbers: ($C_0, \vec{C}$)
- (5b) create function that outputs array of 4 random complex numbers
- (5c) Test function using random A and B arrays compare two ways

In [3]:
@nb.jit(nopython=True)
def inner_product(v,w):
    vx, vy, vz = v
    wx, wy, wz = w
    dot = vx*wx + vy*wy +vz*wz
    return dot

@nb.jit(nopython=True)
def inner_product1(v,w):
    return v[0]*w[0] + v[1]*w[1] +v[2]*w[2]

In [4]:
v = np.array([1,1,1])
w = np.array([1,2,2])
p = inner_product1(v,w)
p

5

In [5]:
##5a: make general matrix maker
#@nb.jit(nopython=True)
def matrix_maker(m0xyz): #array to matrix
    M0, Mx, My, Mz = m0xyz
    
    identity_matrix = np.zeros((2,2), dtype = 'complex128')
    identity_matrix[0,0]= 1
    identity_matrix[1,1] = 1

    pauli_x = np.zeros((2,2), dtype = 'complex128')
    pauli_x[0,1]= 1
    pauli_x[1,0] = 1

    pauli_y = np.zeros((2,2), dtype = 'complex128')
    pauli_y[0,1]= -1j
    pauli_y[1,0] = 1j

    pauli_z = np.zeros((2,2), dtype = 'complex128')
    pauli_z[0,0]= 1
    pauli_z[1,1] = -1
    
    m = np.array([Mx, My, Mz])
    
    pauli_basis = np.array([pauli_x, pauli_y, pauli_z])
    
    m_dot_pauli = Mx*pauli_x + My*pauli_y + Mz*pauli_z
    
    matrix = M0 * identity_matrix + m_dot_pauli
    
    return matrix

In [6]:
a0xyz = np.array([1, 1, 1, 1])
b0xyz = np.array([2, 2, 2, 2])
A = matrix_maker(a0xyz)
B = matrix_maker(b0xyz)
C = A@B
print(A)
print(B)
print(C)

[[2.+0.j 1.-1.j]
 [1.+1.j 0.+0.j]]
[[4.+0.j 2.-2.j]
 [2.+2.j 0.+0.j]]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [7]:
a0xyz = np.array([1, 1, 1, 1])
b0xyz = np.array([2, 2, 2, 2])
A = matrix_maker(a0xyz)
B = matrix_maker(b0xyz)
C = A@B
print(A)
print(B)
print(C)

[[2.+0.j 1.-1.j]
 [1.+1.j 0.+0.j]]
[[4.+0.j 2.-2.j]
 [2.+2.j 0.+0.j]]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [8]:
## takes two arrays and creates a third that can then be used to make a matrix
def calculate_c0xyz(a0xyz, b0xyz): #arrays to array
    A0, Ax, Ay, Az = a0xyz
    B0, Bx, By, Bz = b0xyz
    axyz = np.array([Ax, Ay, Az])
    bxyz = np.array([Bx, By, Bz])
    C0 = A0*B0 + np.tensordot(axyz, bxyz, axes=1)
    cxyz = A0*bxyz + B0*axyz +1j*(np.cross(axyz, bxyz))
    c0xyz = ([C0, cxyz[0], cxyz[1], cxyz[2]])
    return c0xyz

In [9]:
#testing calculate_c0xyz
e0xyz = [1, 1, 1, 1]
f0xyz = [2, 2, 2, 2]
g0xyz = calculate_c0xyz(e0xyz, f0xyz)
print(g0xyz)
G = matrix_maker(g0xyz)
print(G)

[8, (4+0j), (4+0j), (4+0j)]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [10]:
##5b make random number generator
    ## np.random.randn(n) returns a 1D array with n random numbers, so here it makes a 1D array of 4 random numbers (could make a matrix of random numbers or a 2D array i should say, if you give two inputs like np.random.randn(m,n))
    ## and "randn" refers to "random normal" which means it generates numbers from the standard normal distribution (gaussian distribution)
def random_complex_numbers():
    real_parts = np.random.randn(4)
    imaginary_parts = np.random.randn(4)
    complex_numbers = real_parts + 1j*imaginary_parts
    return complex_numbers

random_numbers = random_complex_numbers()
print(random_numbers)

[-0.23563999+0.90690458j  1.34068112+0.06655576j  0.3248179 -0.72715698j
 -0.26107952-0.09349464j]


In [11]:
#take a0xyz and b0xyz and make c0xyz then C
def one_matrix_test(a0xyz, b0xyz): #arrays to matrix
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

#take a0xyz and b0xyz and make A and B then C
def three_matrix_test(a0xyz, b0xyz): #arrays to matrix
    A = matrix_maker(a0xyz)
    B = matrix_maker(b0xyz)
    C = A@B
    return C

In [12]:
#testing
h0xyz = [1, 1, 1, 1]
i0xyz = [2, 2, 2, 2]
C1 = one_matrix_test(h0xyz, i0xyz)
C2 = three_matrix_test(h0xyz, i0xyz)

In [13]:
norm_diff(C1, C2)
print(C1)
print (C2)

[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [14]:
### 5c: testing function using random numbers
def trial_loop(N): 

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()

        #compute density matrix both ways
        L = one_matrix_test(random_numbers1, random_numbers2)
        M = three_matrix_test(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [15]:
N = 100
delta_values, random_numbers_used = trial_loop(N) #run like this to capture outputs

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [7.737793025654259e-17, 7.155609629503253e-17, 9.227683514622779e-17, 8.786087308544193e-17, 4.36906047909336e-17, 1.4312471382881167e-17, 7.262276703934274e-17, 1.4240862566727962e-16, 9.08580938040836e-17, 1.2912311263858319e-16, 9.967015310443018e-17, 6.389845627806323e-17, 9.106164924289454e-17, 7.081770364099569e-17, 5.328079983033522e-17, 6.390627755353813e-17, 7.995367058510766e-17, 7.752248099089427e-17, 6.693625950805396e-17, 8.230496085459113e-17, 8.426968603743959e-17, 8.721558960974458e-17, 5.681702025076339e-17, 8.969485132704177e-17, 7.200908426031401e-17, 6.884436629020803e-17, 1.1286164809872602e-16, 5.728373210297443e-17, 9.330487752464118e-17, 8.020534639390821e-17, 5.0123437291421465e-17, 6.100777709104452e-17, 4.8348123202075906e-17, 5.834831718605362e-17, 6.959481682449377e-17, 5.444480264658945e-17, 4.9519391589671753e-17, 6.361559917210106e-17, 2.6274433565702867e-17, 9.879515167706797e-17, 1.2487609494507401e-16, 8.745462469448317e-17, 5.3866049127

## 6: Create matrix multiplication for $(\mathbf{1} - \rho^{(2)})\rho^{(4)}$
- (6a): create functions that input $(P_0, \vec{P})$ and output $(A_0, \vec{A})$ and $(B_0, \vec{B})$
- (6b): test

In [16]:
## 6a
def array_for_rho4(p0xyz): 
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = ([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

def array_for_rho2(p0xyz):
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = ([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz
#i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

^ i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

In [17]:
## 6b testing
p0xyz = [1, 1, 1, 1]
b0xyz = array_for_rho4(p0xyz) #how to check this
print('ARRAY B:', b0xyz)
rho4 = matrix_maker(b0xyz)
print('RHO4:',rho4)

p0xyz = [1, 1, 1, 1]
a0xyz = array_for_rho2(p0xyz) #how to check this
print('ARRAY A:', a0xyz)
rho2 = matrix_maker(a0xyz)
print('RHO2:', rho2)

ARRAY B: [0.5, 0.5, 0.5, 0.5]
RHO4: [[1. +0.j  0.5-0.5j]
 [0.5+0.5j 0. +0.j ]]
ARRAY A: [0.5, -0.5, -0.5, -0.5]
RHO2: [[ 0. +0.j  -0.5+0.5j]
 [-0.5-0.5j  1. +0.j ]]


## 7: Calculate $ C = (\mathbf{1} - \rho^{(2)})\rho^{(4)} = C_0\mathbf{1} + \vec{C}* \vec{\sigma}$
- (7a): create function that inputs 2 arrays ($P_0^{(2)}, \vec{P^{(2)}}$) and ($P_0^{(4)}, \vec{P^{(4)}}$) and outputs $(C_0, \vec{C})$
- (7b): test

- $\textbf{create_rho2}$ creates an array that can be put into matrix maker to create matrix "rho2"
- $\textbf{create_rho4}$ creates an array that can be put into matrix maker to create matrix "rho4"

- "rho2" is a matrix:  $(\mathbf{1} - \rho^{(2)})$
- "rho4" is a matrix: $\rho^{(4)}$

- so to create rho2 or rho4, need to take p2 or p4 and input it into the correct function, then we will have an array to put into matrix maker to get rho2 and rho4?
..... i am getting lost

In [18]:
#### dont need this: do need this!!!
def rho2_matrix(p0xyz_2):
    rho2_array = array_for_rho2(p0xyz_2)
    rho2 = matrix_maker(rho2_array)
    return rho2

def rho4_matrix(p0xyz_4):
    rho4_array = array_for_rho4(p0xyz_4)
    rho4 = matrix_maker(rho4_array)
    return rho4

In [19]:
##7a
def rho2timesrho4(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    return c0xyz

p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
c10xyz = rho2timesrho4(p0xyz_2, p0xyz_4)
C1 = matrix_maker(c10xyz)
print(c10xyz)
print(C1)

[-0.5, 0j, 0j, 0j]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


COMPARING FUNCTIONS:
- function 1, $\textbf{final_C_1}$ is taking in p2 and p4 arrays, creating a0xyz and b0xyz arrays to then get a third array c0xyz which can go into matrix maker and get the final matrix C
- function 2, $\textbf{final_C_2}$ is taking in p2 and p4 arrays and making rho2 and rho4 matrices to then be multiplied and create matrix C


In [20]:
##7b: comparing functions:
%time
def final_C_1(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C1 = matrix_maker(c0xyz)
    return C1
%time
def final_C_2(p0xyz_2, p0xyz_4):
    rho2 = rho2_matrix(p0xyz_2)
    rho4 = rho4_matrix(p0xyz_4)
    C2 = rho2@rho4
    return C2

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs
CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [21]:
p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
C1 = final_C_1(p0xyz_2, p0xyz_4)
C2 = final_C_2(p0xyz_2, p0xyz_4)
print (C1)
print (C2)

[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


In [22]:
### testing with random numbers
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        
        L = final_C_1(random_numbers1, random_numbers2)
        M = final_C_2(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [23]:
N = 100 
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [9.263809991785739e-17, 7.233257917225715e-17, 9.159067542536093e-17, 3.53305750354308e-17, 8.205145229403647e-17, 5.045628811787607e-17, 6.99309207598833e-17, 7.972904122495579e-17, 1.0883062718477378e-16, 8.355678537186085e-17, 5.99510416279121e-17, 4.705102752096247e-17, 8.509338779835587e-17, 6.275175177200905e-17, 6.714563847175419e-17, 8.527322061518847e-17, 9.239405701344953e-17, 5.390017838665817e-17, 7.117196090083872e-17, 5.812429518111673e-17, 4.8425594960086704e-17, 4.796040889376994e-17, 7.515175897812896e-17, 6.218339855234311e-17, 4.4081145540291934e-17, 8.217999142573827e-17, 1.189924520021774e-16, 3.2577023992295863e-17, 5.7160991930098e-17, 1.2290277789506445e-16, 5.863883353409802e-17, 9.23467091239661e-17, 4.50181286920911e-17, 4.513721271041562e-17, 7.835056772646115e-17, 8.268779252371251e-17, 1.1259554753322804e-16, 8.332465227760237e-17, 9.052127931824318e-17, 1.2018638319671056e-16, 7.160035006412382e-17, 7.45477655006095e-17, 1.882922096111126e-1

## 8: Traces
- (8a): find $D = C + \mathrm{trc}(C)\mathbf{1} = D_0\mathbf{1} + \vec{D} * \vec{\sigma}$
- (8b): Test the equivalent of $(D_0, \vec{D})$ and $D = C + \mathrm{trc}(C)\mathbf{1}$

In [24]:
##8a
c0xyz = rho2timesrho4(p0xyz_2, p0xyz_4) #arrays were generated in cell 18
print('check C array:', c0xyz)

check C array: [-0.5, 0j, 0j, 0j]


In [25]:
C = final_C_1(p0xyz_2, p0xyz_4) #can use either function c1 or c2 both make the matrix c
print ('check matrix C:', C)

check matrix C: [[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


In [26]:
trC = np.trace(C)
print('trace of C=', trC, '2C_0=', 2*c0xyz[0])

identity_matrix = np.zeros((2,2), dtype = 'complex128')
identity_matrix[0,0]= 1
identity_matrix[1,1] = 1

trace of C= (-1+0j) 2C_0= -1.0


In [27]:
D = C + np.trace(C)*identity_matrix
print('check matrix D:', D)

check matrix D: [[-1.5+0.j  0. +0.j]
 [ 0. +0.j -1.5+0.j]]


In [28]:
#making matrix D another way: takes matrix D and produces d0xyz?
D2 = C + 2*c0xyz[0]
print (D)
print (D2)

[[-1.5+0.j  0. +0.j]
 [ 0. +0.j -1.5+0.j]]
[[-1.5+0.j -1. +0.j]
 [-1. +0.j -1.5+0.j]]


## 9: Creating the real functions!
- create functions to calculate E and F:
- $E = \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big] $
    - $E = \rho^{(3)} \big[C + \mathrm{tr}(C)\big] $
- $F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big] $
    - $F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[C + \mathrm{tr}(C)\big] $

#### THE BIG IDEA:

so the idea will be to slowly build up to function F
- first we create $(\mathbf{1} -\rho^{(2)})\rho^{(4)}$, which we call C
    - call $\mathbf{rho2timesrho4}$(p0xyz_2, p0xyz_4) and it returns c0xyz
    - the you would do matrix C = $\textbf{matrix_maker}$(c0xyz)
    - OR call $\textbf{final_C_1}$(p0xyz_2, p0xyz_4) 
    - ***** use final c1 or final c2?

- then we will create $(\mathbf{1} -\rho^{(2)})\rho^{(4)}+ \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)$, which we will call D (just adding trace * identity onto C)
- then we will multiply matrix $\rho^{(3)}$, which we will call E
- lastly, we will multiply matrix $(\mathbf{1} -\rho^{(1)})$, which is then F

- Input: $P^{(1)}$, $P^{(2)}$, $P^{(3)}$, $P^{(4)}$, which I call p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4
- (10a): Output: $(2\mathrm{Re}[F_0], 2\mathrm{Re}[\vec{F}])$

In [29]:
# define final_C_1 if i change the function name...
# define identity_matrix
# need matrix_maker
# need to define P vectors
p0xyz_1 = [1, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]

In [30]:
## need new name for this
def matrix_mult_arrays(m10xyz, m20xyz): #arrays to array
    M10, M1x, M1y, M1z = m10xyz
    M20, M2x, M2y, M2z = m20xyz
    m1xyz = np.array([M1x, M1y, M1z])
    m2xyz = np.array([M2x, M2y, M2z])
    M30 = M10*M20 + np.tensordot(m1xyz, m2xyz, axes=1)
    m3xyz = M10*m2xyz + M20*m1xyz +1j*(np.cross(m1xyz, m2xyz))
    m30xyz = ([M30, m3xyz[0], m3xyz[1], m3xyz[2]])
    return m30xyz

In [31]:
c0xyz =[2, 1, 1, 1]
d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
print (d0xyz)

[6, 1, 1, 1]


In [32]:
## okay so we are going to make 2 functions to calculate F.... 
## one does the RHS which does matrix all the way
## the other does the LHS which keeps arrays all the way

#this function is for testing
def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = final_C_1(p0xyz_2, p0xyz_4)
    #print ('C=', C)
    D = C + np.trace(C)*identity_matrix
    #print ('D=', D)
    E = matrix_maker(p0xyz_3)@D
    #print ('E=', E)
    F = matrix_maker(p0xyz_1)@E
    return (F)

#this function will go into code
def F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    #convert p's to a and b - what function does this? array_for_rho2 and array_for_rho4
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
   
    #multiply a and b
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)

    #add d
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
   
    #multiply p0xyz_3 to get e
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
   
    #multiply p0xyz_1 to get f
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
   
    #make array matrix
    F = matrix_maker(f0xyz)
    return (F)

In [33]:
p0xyz_1 = [1, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]

print ('INPUT P ARRAYS')
print('P1=', p0xyz_1)
print('P2=', p0xyz_2)
print('P3=', p0xyz_3)
print('P4=', p0xyz_4)

print ('CONVERTED ARRAYS')
print('a0xyz=', array_for_rho2(p0xyz_2))
print('b0xyz=', array_for_rho4(p0xyz_4))
print('c0xyz=', matrix_mult_arrays(a0xyz, b0xyz))

print ('MATRICES')
print ('A=',rho2_matrix(p0xyz_2))
print ('B=', rho4_matrix(p0xyz_4))

print ('F made with matrices', F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4))

print ('F made with arrays', F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4))

# compare both ways
FM = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
FA = F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print(type(FA))
print(type(FM))

INPUT P ARRAYS
P1= [1, 1, 1, 1]
P2= [1, 1, 1, 1]
P3= [1, 1, 1, 1]
P4= [1, 1, 1, 1]
CONVERTED ARRAYS
a0xyz= [0.5, -0.5, -0.5, -0.5]
b0xyz= [0.5, 0.5, 0.5, 0.5]
c0xyz= [-0.5, 0j, 0j, 0j]
MATRICES
A= [[ 0. +0.j  -0.5+0.5j]
 [-0.5-0.5j  1. +0.j ]]
B= [[1. +0.j  0.5-0.5j]
 [0.5+0.5j 0. +0.j ]]
F made with matrices [[-9.+0.j -3.+3.j]
 [-3.-3.j -3.+0.j]]
F made with arrays [[-9.+0.j -3.+3.j]
 [-3.-3.j -3.+0.j]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [34]:
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways
        L = F_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = F_array_to_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [35]:
N = 100
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [7.512404146545926e-17, 1.7013631238659108e-16, 7.310550874416207e-17, 8.029047045488053e-17, 5.896415409942333e-17, 6.73798664247159e-17, 1.0955819466571806e-16, 5.828149456943543e-17, 6.90630804033199e-17, 6.050976631073422e-17, 1.2970063178744566e-16, 1.1388388417670946e-16, 1.514329217146451e-16, 8.621430786636557e-17, 1.1056316221952153e-16, 1.6140815045687657e-16, 1.7747373907086902e-16, 1.5469981585944592e-16, 1.7431813545835284e-16, 9.905917246765341e-17, 1.0139347192349922e-16, 9.198654461847489e-17, 1.1200385761498751e-16, 1.716833517054249e-16, 2.5698923925944698e-17, 1.2442787119394492e-16, 9.196461294288831e-17, 1.6210562186633933e-16, 1.6330787253956593e-16, 1.0594301323487695e-16, 1.2864250763425256e-16, 5.5361721453329416e-17, 9.584644030000067e-17, 7.776751250948829e-17, 1.0104887976813052e-16, 6.230403371419816e-17, 8.350322398503736e-17, 1.668991553824765e-16, 9.324325988069793e-17, 1.4079904622580723e-16, 1.406357670948701e-16, 9.741903731692327e-17, 6

## 10: Function from Bennett et al
- make function that inputs the 4 p arrays and outputs an array of REAL NUMBERS so output: $(2\mathrm{Re}[F_0], 2\mathrm{Re}[\vec{F}])$

testing complex conjugate stuff:

In [36]:
# found np.conjugate(): function that takes the complex conjugate
z = 3 + 2*1j
z_conj = np.conjugate(z)
print(z, z_conj)

(3+2j) (3-2j)


In [37]:
z = [1, -1j, 1j, 1]
z_conj = np.conjugate(z)
print(z, z_conj)

[1, (-0-1j), 1j, 1] [ 1.-0.j -0.+1.j  0.-1.j  1.-0.j]


In [38]:
def F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    a0xyz = array_for_rho2(p0xyz_2) # (1-rho) 
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    return (f0xyz)

def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = final_C_1(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    E = matrix_maker(p0xyz_3)@D
    F = matrix_maker(p0xyz_1)@E
    return (F)

p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
f0xyz_conj = np.conjugate(f0xyz)

print('ARRAY STUFF')
print('f0xyz=',f0xyz)
print('f0xyz_conj=',f0xyz_conj)
Re_f0xyz = f0xyz + f0xyz_conj
print('Re_f0xyz', Re_f0xyz)

Re_F_made_w_arrays = matrix_maker(Re_f0xyz)
print(Re_F_made_w_arrays)

F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
F_conj = np.conjugate(F)

print('MATRIX STUFF')
print('F=',F)
print('F_conj=',F_conj)
Re_F = F + F_conj
print('Re_F=',Re_F)

ARRAY STUFF
f0xyz= [(-3+1j), (-2+0j), (-1-1j), (-2+0j)]
f0xyz_conj= [-3.-1.j -2.-0.j -1.+1.j -2.-0.j]
Re_f0xyz [-6.+0.j -4.+0.j -2.+0.j -4.+0.j]
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
MATRIX STUFF
F= [[-5.+1.j -3.+1.j]
 [-1.-1.j -1.+1.j]]
F_conj= [[-5.-1.j -3.-1.j]
 [-1.+1.j -1.-1.j]]
Re_F= [[-10.+0.j  -6.+0.j]
 [ -2.+0.j  -2.+0.j]]


In [39]:
## 10a:
# the funciton as is outputs the F matrix, we want to take that and
# take the real part and print out the array of that
def Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

def arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [40]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]


Re_f0xyz = Fvvsc_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
Fvvsc_from_array = matrix_maker(Re_f0xyz)
print('Fvvsc_array PRODUCES:')
print(Fvvsc_from_array)

Fvvsc_arr_2_mat = arr_2_mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('arr_2_mat PRODUCES:')
print(Fvvsc_arr_2_mat)

F_made_with_matrices = Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
print('Fvvsc_matrix PRODUCES:')
print(F_made_with_matrices)

Fvvsc_array PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
arr_2_mat PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]
Fvvsc_matrix PRODUCES:
[[-10.+0.j  -4.+2.j]
 [ -4.-2.j  -2.+0.j]]


In [41]:
## testing
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways   
        L = arr_2_mat(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = Fvvsc_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [42]:
N = 2
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [6.195633082179349e-17, 1.1210144346485765e-16]
RANDOM NUMBERS USED: [(array([-0.30398362-1.21567397j, -1.18399442+0.23150219j,
       -0.74122369+0.05130099j,  0.21253135-1.31972073j]), array([-0.31641717-0.20072515j, -0.37633965+0.03707373j,
        0.54811909+0.87250989j,  0.32070658-0.45468928j]), array([-0.32000149-2.94380012j,  1.03044116-0.27681448j,
       -2.09752   +0.48384079j, -0.58108876+0.05248383j]), array([-1.01119134-0.00484567j,  0.01300533+1.44424794j,
       -1.32326313-0.88728896j, -1.35698136+1.2936146j ])), (array([ 0.18121859-0.01699539j,  1.19273147+1.34261863j,
       -0.55361602-1.12890613j,  1.15493894+0.56382324j]), array([ 1.06704148-1.1410329j , -1.40515539-1.68250812j,
       -1.20714354+0.24045308j,  0.60631589+0.63388982j]), array([-0.6914305 +0.66463108j, -0.94776121-0.69447651j,
        0.64193922-0.49550693j,  0.2445302 -1.46572563j]), array([-1.85820979+0.07821651j,  0.39784814-1.56071404j,
        0.32916461+0.31636184j, -2.22092848+

In [43]:

#two ways to make final F
def F_array(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    return (f0xyz)
#note: can probably refine F_array_to_matrix function to just call F_array and then matrix maker
def F_array_to_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    a0xyz = p0xyz2_to_a0xyz(p0xyz_2)
    b0xyz = p0xyz4_to_b0xyz(p0xyz_4)
    c0xyz = matrix_mult_arrays(a0xyz, b0xyz)
    d0xyz = [c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]]
    e0xyz = matrix_mult_arrays(p0xyz_3, d0xyz)
    f0xyz = matrix_mult_arrays(p0xyz_1, e0xyz)
    F = matrix_maker(f0xyz)
    return (F)

def F_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    C = p0xyz2_p0xyz4_to_C(p0xyz_2, p0xyz_4)
    D = C + np.trace(C)*identity_matrix
    E = matrix_maker(p0xyz_3)@D
    F = matrix_maker(p0xyz_1)@E
    return (F)

# RENAMING THINGS-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [44]:
#matrix 1 array, matrix 2 array --> output matrix 3 array

######### KEY:
######### takes two, 4 component vectors each corresponding to a matrix
######### multiplies the "matrices" but keeps them as vector/array representations the whole time
######### outputs the components for the resulting matrix
def multiply_matrices(M1components, M2components): 
    M10, M1x, M1y, M1z = M1components
    M20, M2x, M2y, M2z = M2components
    m1xyz = np.array([M1x, M1y, M1z])
    m2xyz = np.array([M2x, M2y, M2z])
    M30 = M10*M20 + np.tensordot(m1xyz, m2xyz, axes=1)
    m3xyz = M10*m2xyz + M20*m1xyz +1j*(np.cross(m1xyz, m2xyz))
    M3components = np.array([M30, m3xyz[0], m3xyz[1], m3xyz[2]])
    return M3components

#functions to take p arrays and make general a and b, so that we can do multiplication noramllly

######### KEY:
######### this function generalizes the specific array that is (identity minus rho)...
######### it takes the 4 component vector/array that would create a matrix that is in the form (1-rho)
######### makes it into more generalized components "a0xyz"
def generalize_IDrho(IDrho_array):
    P0, Px, Py, Pz = IDrho_array
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = np.array([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz #makes array to be put into matrix maker to create (ID-rho)

######### KEY:
######### this function generalizes the specific array that is just rho...
######### it takes the 4 component vector/array that would create a matrix that is in the form rho
######### makes it into more generalized components "b0xyz"
def generalize_rho(rho_array):
    P0, Px, Py, Pz = rho_array
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = np.array([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

######### KEY:
######### this function generalizes then multiplies:
######### first input is rho matrix that is identity minus rho (1-rho)
######### second input is rho matrix that is solo (rho)
######### takes generalized versions and multiplies them in the order (1-rho)rho
def generalize_then_mult(IDrho_array, rho_array):
    a0xyz = generalize_IDrho(IDrho_array)
    b0xyz = generalize_rho(rho_array)
    c0xyz = multiply_matrices(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

######### KEY:
######### this function generalizes then multiplies:
######### first input is rho matrix that is identity minus rho (1-rho)
######### second input is rho matrix that is solo (rho)
######### NOTE THE DIFFERENCE:
######### this function takes generalized versions and multiplies them in the order rho(1-rho)
def generalize_then_mult_term2(IDrho_array, rho_array):
    a0xyz = generalize_IDrho(IDrho_array)
    b0xyz = generalize_rho(rho_array)
    c0xyz = multiply_matrices(b0xyz, a0xyz)
    C = matrix_maker(c0xyz)
    return C

In [45]:
#two ways to make final F (term 1) 
#NOTE: these are all not the final Fvvsc, because they have not yet added the harmonic conjugate

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it outputs the resulting matrix for this term, but as its ARRAY/VECTOR representation
def F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # (1-rho2)
    a0xyz = generalize_IDrho(p0xyz_2)
    
    # rho4
    b0xyz = generalize_rho(p0xyz_4)
    
    # (1-rho2)rho4 ....aka....
    # a0xyz * b0xyz
    c0xyz = multiply_matrices(a0xyz, b0xyz)
    
    # (1-rho2)rho4 + tr(-)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])
    
#now need to generalize rho3 and rho1 arrays
    
    # (1-rho1)
    g0xyz = generalize_IDrho(p0xyz_1)
    
    # rho3
    h0xyz = generalize_rho(p0xyz_3)
    
    # e0xyz = rho3[(1-rho2)rho4 + tr(-)] .... aka.....
    # e0xyz = h0xyz * d0xyz
    e0xyz = multiply_matrices(h0xyz, d0xyz)
    
    # f0xyz = (1-rho1)rho3[(1-rho2)rho4 + tr(-)] .... aka.....
    # f0xyz = g0xyz * h0xyz * d0xyz .... aka.....
    # f0xyz = g0xyz * e0xyz
    f0xyz = multiply_matrices(g0xyz, e0xyz)
    return (f0xyz)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it does this by using the components created from "F_components_term1" function then putting it into matrix_maker
def F_array_to_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F = matrix_maker(f0xyz)
    return (F)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the first term in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)]
######### it outputs the resulting MATRIX for this term
def F_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # C = (1-rho2)rho4 ..... or.....
    # C = A*B 
    C = generalize_then_mult(p0xyz_2, p0xyz_4)
    
    # D = (1-rho2)rho4 + tr((1-rho2)rho4)
    # D = C + tr(C)*I
    D = C + np.trace(C)*identity_matrix
    
    # need to generalize rho3 and rho1 arrays
    # (1-rho1) 
    g0xyz = generalize_IDrho(p0xyz_1)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_3)
    
    # E = rho3[(1-rho2)rho4 + tr((1-rho2)rho4)]
    # E = rho3*[C + tr(C)*I]
    # E = rho3*D
    E = matrix_maker(h0xyz)@D
    
    # F = (1-rho1)rho3[(1-rho2)rho4 + tr((1-rho2)rho4)]
    # F = (1-rho1)rho3*[C + tr(C)*I]
    # F = (1-rho1)rho3*D
    # F = (1-rho1)*E
    F = matrix_maker(g0xyz)@E
    return (F)

## 11: Repeat but for the other term
- (11a): create another function: switch 2 and 4 and switch 1 and 3
$F = (\mathbf{1} - \rho^{(1)}) \rho^{(3)}\big[(\mathbf{1} -\rho^{(2)})\rho^{(4)} + \mathrm{tr}\big((\mathbf{1} -\rho^{(2)})\rho^{(4)}\big)\big]  - \rho^{(1)}(\mathbf{1} - \rho^{(3)})\big[\rho^{(2)}(\mathbf{1} -\rho^{(4)}) + \mathrm{tr}\big(\rho^{(2)}(\mathbf{1} -\rho^{(4)})\big)\big] + \mathrm{h.c.} $
- (11b): test it
(we will do this for all the terms in A.11)

In [46]:
######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it outputs the resulting matrix for this term, but as its ARRAY/VECTOR representation
def F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4): #same function as before but minus making the matrix at the end
    # (1-rho4)
    a0xyz = generalize_IDrho(p0xyz_4)
    
    ######should i do it like a is always the lower rho number, or should 
    ######i do it like a is always the first matrix
    ######or like a is always the identity one (this is what I have now)--- DECIDED TO KEEP THIS WAY
    
    # rho2
    b0xyz = generalize_rho(p0xyz_2)
    
    # rho2(1-rho4)
    c0xyz = multiply_matrices(b0xyz, a0xyz)
    
    # rho2(1-rho4) + tr(-)
    d0xyz = np.array([c0xyz[0] + 2*c0xyz[0], c0xyz[1], c0xyz[2], c0xyz[3]])

    ###generalize rho3 and rho1 
    
    # (1-rho3) 
    g0xyz = generalize_IDrho(p0xyz_3)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_1)

    # e0xyz = (1-rho3)[rho2(1-rho4) + tr(-)]
    # e0xyz = g0xyz * d0xyz
    e0xyz = multiply_matrices(g0xyz, d0xyz)
    
    # f0xyz = rho1(1-rho3)[rho2(1-rho4) + tr(-)]
    # f0xyz = h0xyz * g0xyz * d0xyz
    # f0xyz = h0xyz * e0xyz
    f0xyz = multiply_matrices(h0xyz, e0xyz)
    return (f0xyz)

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it does this by using the components created from "F_components_term2" function then putting it into matrix_maker
def F_array_to_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F = matrix_maker(f0xyz)
    return (F)

                     
######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the second term in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)]
######### it outputs the resulting MATRIX for this term
def F_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    # C = rho2(1-rho4) ..... or.....
    # C = B*A 
    C = generalize_then_mult_term2(p0xyz_4, p0xyz_2) 
    
    # D = rho2(1-rho4) + tr(rho2(1-rho4))
    # D = C + tr(C)*I
    D = C + np.trace(C)*identity_matrix
    
    # need to generalize rho3 and rho1 arrays
    # (1-rho3) 
    g0xyz = generalize_IDrho(p0xyz_3)
    
    # rho1
    h0xyz = generalize_rho(p0xyz_1)
    
    # E = (1-rho3)[rho2(1-rho4) + tr(rho2(1-rho4))]
    # E = (1-rho3)*[C + tr(C)*I]
    # E = (1-rho3)*D
    E = matrix_maker(g0xyz)@D
    
    # F = rho1(1-rho3)[rho2(1-rho4) + tr(rho2(1-rho4))]
    # F = rho1(1-rho3)*[C + tr(C)*I]
    # F = rho1(1-rho3)*D
    # F = rho1*E
    F = matrix_maker(h0xyz)@E
    return (F)

## Fvvsc with new function names

In [47]:
#NEW FUNCTION NAMES FVVSC TERM 1
# now adding in the harmonic conjugates

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the FIRST TERM in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)] using F_components_term1
######### then it adds the conjugate of this array, adds it to the original array, and outputs the real array
def Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the FIRST TERM in Bennett et al. A.11: (1-rho1)rho3[(1-rho2)rho4 + tr(-)] using F_matrix_term1
######### then it adds the harmonic conjugate (conjugate and transpose) of this matrix, adds it to the original matrix, and outputs the hermitian matrix
def Fvvsc_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

######## KEY:
######## takes output array from Fvvsc_components_term1 and makes matrix
def Fvvsc_arr2mat_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

In [48]:
#NEW FUNCTION NAMES FVVSC TERM 2

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the SECOND TERM in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)] using F_components_term2
######### then it adds the harmonic conjugate of this array, adds it to the original array, and outputs the real array
def Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    f0xyz = F_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    f0xyz_conj = np.conjugate(f0xyz)
    Re_f0xyz = f0xyz + f0xyz_conj
    return Re_f0xyz

######### KEY:
######### this function takes four vectors: each a 4 component array to represent a matrix rho
######### it calculates the arrays for the SECOND TERM in Bennett et al. A.11: rho1(1-rho3)[rho2(1-rho4) + tr(-)] using F_matrix_term2
######### then it adds the harmonic conjugate (conjugate and transpose) of this matrix, adds it to the original matrix, and outputs the hermitian matrix
def Fvvsc_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    F = F_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    F_conj = np.conjugate(F)
    F_dagger = np.transpose(F_conj)
    Fvvsc = F + F_dagger
    return Fvvsc

######## KEY:
######## takes output array from Fvvsc_components_term2 and makes matrix
def Fvvsc_arr2mat_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    Re_f0xyz = Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    Fvvsc_from_array = matrix_maker(Re_f0xyz)
    return Fvvsc_from_array

## FULL Fvvsc, BOTH TERMS

In [49]:
#now for the full term!

######## KEY:
######## this creates the full Bennet A.11 equation in ARRAY/VECTOR/COMPONENT form:
######## it does this by taking in those 4 arrays corresponding to 4 matrices
######## and then it uses Fvvsc_components_term1 and Fvvsc_components_term2 
######## to output the full array/vector representatation of the full Fvvsc term
def Fvvsc_components(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_components_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_components_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

######## KEY:
######## this creates the full Bennet A.11 equation in MATRIX form: 
######## it does this by taking in those 4 arrays corresponding to 4 matrices
######## and then it uses Fvvsc_matrix_term1 and Fvvsc_matrix_term2 
######## to output the full MATRIX of the full Fvvsc term
def Fvvsc_matrix(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_matrix_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_matrix_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

######## KEY:
######## uses Fvvsc_arr2mat_term1 and Fvvsc_arr2mat_term2 to make the full MATRIX of the full Fvvsc term
def Fvvsc_arr2mat(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4):
    term1 = Fvvsc_arr2mat_term1(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    term2 = Fvvsc_arr2mat_term2(p0xyz_1, p0xyz_2, p0xyz_3, p0xyz_4)
    return term1 + term2

In [50]:
p0xyz_1 = [1j, 1, 1, 1]
p0xyz_2 = [1, 1, 1, 1]
p0xyz_3 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, -1j]

def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        random_numbers3 = random_complex_numbers()
        random_numbers4 = random_complex_numbers()

        #compute density matrix both ways   
        L = Fvvsc_arr2mat(random_numbers1, random_numbers2, random_numbers3, random_numbers4)
        M = Fvvsc_matrix(random_numbers1, random_numbers2, random_numbers3, random_numbers4)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2, random_numbers3, random_numbers4)) #called it something different because i was trying to append random numbers to itslef and it said no
   
    return delta_values, random_numbers_used

In [51]:
N = 100
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [1.8742518982651585e-16, 6.948891749453128e-17, 2.564059150507208e-16, 1.0772848063330413e-16, 3.165248852471336e-16, 9.068710620801781e-17, 1.3357343538020038e-16, 1.5050704554056441e-16, 1.0899370627804147e-16, 1.201416588789944e-16, 2.3024550143720583e-16, 1.9163127598861702e-16, 2.0860965928133816e-16, 9.454421632310396e-17, 7.12715029061778e-17, 1.505577674257844e-16, 7.802704200336042e-17, 1.0453997770856619e-16, 6.900349411419552e-17, 8.92464528116117e-17, 4.135137132720802e-17, 4.7835063096146814e-17, 8.580391077037574e-17, 1.5232373374560407e-16, 2.9856467117203985e-16, 6.766111830852544e-17, 1.544612727711124e-16, 1.2953761615879894e-16, 1.2080207121910602e-16, 9.241155835247071e-17, 3.1584871965427974e-17, 9.183430020281473e-17, 1.036231242150687e-16, 6.296796245486137e-17, 9.614153411860396e-17, 3.9047376365764236e-16, 2.3306832906706984e-16, 1.1799907779065305e-16, 6.175765344299921e-17, 1.0344008660737806e-16, 5.1383487693963157e-17, 1.3139727715408119e-16, 